# Deep Hedging — Clean Study Notebook

This notebook is a cleaned version of the original.  
It uses our helper files:

- `deephedge/networks.py` (MLP, NTBN)
- `deephedge/derivatives.py` (NegativeEuropeanOption, CallSpread)
- `deephedge/train.py` (build/train/save/load)
- `deephedge/evaluate.py` (pricing, EU, trade stats, NT bands, plots)

Sections:
1. Setup & Config  
2. Train models (MLP, NTBN)  
3. Study prices & expected utility  
4. Trading measures  
5. No-transaction bands (NTBN)  
6. Call spread study  
7. Buyer vs Writer  


In [ ]:
import os
import torch

from pfhedge.instruments import HestonStock
from deephedge.networks import MLPHedge, NoTransactionBandNet
from deephedge.derivatives import NegativeEuropeanOption, CallSpread
from deephedge.train import build_hedger, train_model, load_or_build
from deephedge.evaluate import (
    indifference_price, expected_utility_from_wealth,
    trade_frequency, average_shares_traded, summarize_trading_stats,
    empirical_nt_bands_from_hedge, plot_pnl_hist, plot_trade_frequency_per_time, plot_nt_bands
)

# Config
SEED = 42
DEVICE = "auto"
FEATURES = ["log_moneyness", "time_to_maturity", "volatility", "prev_hedge"]

RUNS_DIR = "runs"
os.makedirs(RUNS_DIR, exist_ok=True)

CKPT_MLP   = os.path.join(RUNS_DIR, "mlp.pt")
CKPT_NTBN  = os.path.join(RUNS_DIR, "ntbn.pt")

ul = HestonStock()
S0 = 100.0
INIT_STATE = (S0,)

def set_seed(seed=SEED):
    import random, numpy as np
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    random.seed(seed)
    np.random.seed(seed)

set_seed(SEED)


In [ ]:
# Training derivative: Negative European option (buyer)
deriv_train = NegativeEuropeanOption(ul, strike=100.0, maturity=1.0)

# (A) MLP Hedge
model_mlp = MLPHedge(in_features=len(FEATURES), hidden=128, depth=3)
hedger_mlp = load_or_build(CKPT_MLP, model_mlp, FEATURES, device=DEVICE)

out_mlp = train_model(
    hedger_mlp, deriv_train,
    n_paths=50_000,
    n_epochs=10,
    init_state=INIT_STATE,
    seed=SEED,
    save_path=CKPT_MLP,
    save_meta={"features": FEATURES, "model": "MLPHedge"}
)
print("MLP checkpoint:", out_mlp["save_path"])

# (B) NTBN
model_ntbn = NoTransactionBandNet(deriv_train)
hedger_ntbn = load_or_build(CKPT_NTBN, model_ntbn, FEATURES, device=DEVICE)

out_ntbn = train_model(
    hedger_ntbn, deriv_train,
    n_paths=50_000,
    n_epochs=10,
    init_state=INIT_STATE,
    seed=SEED,
    save_path=CKPT_NTBN,
    save_meta={"features": FEATURES, "model": "NoTransactionBandNet"}
)
print("NTBN checkpoint:", out_ntbn["save_path"])


In [ ]:
# Price
price_mlp  = indifference_price(hedger_mlp, deriv_train, n_paths=20_000, init_state=INIT_STATE)
price_ntbn = indifference_price(hedger_ntbn, deriv_train, n_paths=20_000, init_state=INIT_STATE)
print(f"MLP price: {price_mlp:.6f} | NTBN price: {price_ntbn:.6f}")

# Expected utility
# Needs terminal wealth W_T, which you already compute in your notebook.
# Replace the placeholders below with your rollout output.

# --- PLACEHOLDER (replace with your own W_T) ---
W_T_mlp  = torch.randn(20000) * 0.1
W_T_ntbn = torch.randn(20000) * 0.1

EU_mlp  = expected_utility_from_wealth(W_T_mlp,  utility="exp", gamma=1.0)
EU_ntbn = expected_utility_from_wealth(W_T_ntbn, utility="exp", gamma=1.0)
print(f"EU (exp, gamma=1.0): MLP={float(EU_mlp):.6f} | NTBN={float(EU_ntbn):.6f}")


In [ ]:
# Needs hedge paths. Replace placeholders with your rollout results.

# --- PLACEHOLDER (replace with your hedges) ---
n_paths, n_steps = 2000, 50
hedge_mlp  = torch.cumsum(torch.randn(n_paths, n_steps) * 0.01, dim=1)
hedge_ntbn = torch.cumsum(torch.randn(n_paths, n_steps) * 0.01, dim=1)

stats_mlp  = summarize_trading_stats(hedge_mlp)
stats_ntbn = summarize_trading_stats(hedge_ntbn)
print("MLP:", stats_mlp)
print("NTBN:", stats_ntbn)

_ = plot_trade_frequency_per_time(hedge_ntbn, title="NTBN | avg |Δhedge| per time step")


In [ ]:
# Needs spot + hedge paths. Replace placeholders with your rollout results.

# --- PLACEHOLDER ---
spot_fake = torch.linspace(60, 140, steps=n_steps).repeat(n_paths, 1) + torch.randn(n_paths, n_steps)
bands_emp = empirical_nt_bands_from_hedge(spot_fake, hedge_ntbn, n_bins=41, quantile=0.05)
_ = plot_nt_bands(bands_emp, title="Empirical NT bands (from hedge/spot paths)")

# Optional: Direct NTBN outputs on a grid
def ntbn_bands_from_model(derivative, model_ntbn, S_grid, t_indices=(0,)):
    model_ntbn.eval()
    with torch.no_grad():
        lowers, uppers = [], []
        device = next(model_ntbn.parameters()).device
        for t in t_indices:
            lm = torch.log(S_grid / derivative.strike).to(device)
            tau = derivative.time_to_maturity(time_step=t).to(device)
            if tau.dim() > 1: tau = tau[0, 0].repeat_as(lm)
            vol = derivative.volatility(time_step=t).to(device)
            if vol.dim() > 1: vol = vol[0, 0].repeat_as(lm)
            prev = torch.zeros_like(lm)
            x = torch.stack([lm, tau, vol, prev], dim=-1)
            bands = model_ntbn(x)
            lowers.append(bands[:, 0].cpu())
            uppers.append(bands[:, 1].cpu())
        return torch.stack(lowers), torch.stack(uppers)

S_grid = torch.linspace(60, 140, steps=81)
lo_grid, up_grid = ntbn_bands_from_model(deriv_train, model_ntbn, S_grid, t_indices=(0,))
print("Direct NTBN bands at t=0 computed.")


In [ ]:
deriv_cs = CallSpread(ul, strike_long=95.0, strike_short=110.0, maturity=1.0)

price_cs_mlp  = indifference_price(hedger_mlp,  deriv_cs, n_paths=20_000, init_state=INIT_STATE)
price_cs_ntbn = indifference_price(hedger_ntbn, deriv_cs, n_paths=20_000, init_state=INIT_STATE)
print(f"CallSpread prices — MLP: {price_cs_mlp:.6f} | NTBN: {price_cs_ntbn:.6f}")

In [ ]:
deriv_buyer  = NegativeEuropeanOption(ul, strike=100.0, maturity=1.0)  # buyer
deriv_writer = NegativeEuropeanOption(ul, strike=100.0, maturity=1.0)  # writer = short payoff in your class

price_buyer_mlp  = indifference_price(hedger_mlp,  deriv_buyer,  n_paths=20_000, init_state=INIT_STATE)
price_writer_mlp = indifference_price(hedger_mlp,  deriv_writer, n_paths=20_000, init_state=INIT_STATE)

price_buyer_ntbn  = indifference_price(hedger_ntbn, deriv_buyer,  n_paths=20_000, init_state=INIT_STATE)
price_writer_ntbn = indifference_price(hedger_ntbn, deriv_writer, n_paths=20_000, init_state=INIT_STATE)

print(f"Buyer vs Writer (MLP):  buyer={price_buyer_mlp:.6f} | writer={price_writer_mlp:.6f}")
print(f"Buyer vs Writer (NTBN): buyer={price_buyer_ntbn:.6f} | writer={price_writer_ntbn:.6f}")
